In [128]:
from openai import OpenAI
import autogen
from dotenv import load_dotenv
from typing import List, Dict, Annotated, Tuple, Union

load_dotenv('../.env')

assistant = autogen.AssistantAgent(
    name="assistant",
    llm_config={
        "cache_seed": 42,
        "temperature": 0
    }
)

class HumanReadbleResponseAgent(autogen.AssistantAgent):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, 
                         llm_config=False, 
                         system_message="""You are a logger that should reply with a summary of the conversation so far. 
Use the previous messages when creating the summary. 

Your reply should follow this format: 

[Summary of what has happened so far, aimed at a human end user]

[TERMINATE/CONTINUE]

The end of your response should be either "TERMINATE" or "CONTINUE".
Reply "TERMINATE" if the task has been solved at full satisfaction. 
Otherwise, reply "CONTINUE", or the reason why the task is not solved yet.
Make sure "TERMINATE" and "CONTINUE" are in capital letters.  
""", 
                         **kwargs)
        self.register_reply([autogen.Agent, None], HumanReadbleResponseAgent.produce_response)
    
    def produce_response(
        self,
        messages: List[Dict] = [],
        sender=None,
        config=None,
    ) -> Tuple[bool, Union[str, Dict, None]]:
        client = OpenAI()
        stream = client.chat.completions.create(
            model="gpt-4",
            messages=messages,
            stream=True,
        )
        response = ''
        for chunk in stream:
            if chunk.choices[0].delta.content is not None:
                response += chunk.choices[0].delta.content
                print(chunk.choices[0].delta.content, end="")
        return True, response

human_readble_assistant = HumanReadbleResponseAgent(name='human_readble_assistant')

user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
    code_execution_config={
        "work_dir": "coding",
        "use_docker": False,  
    },
)

@user_proxy.register_for_execution()
@assistant.register_for_llm(description="Calculator for doing division.")
def division_calculator(
    dividend: Annotated[int, "The number that is being divided"],
    divisor: Annotated[int, "The number that the dividend being divided by"]
) -> str: # return type must be either str, BaseModel or serializable by json.dumps()
  if divisor == 0:
    return 'Division by 0 is not allowed!'
  return f'{dividend / divisor}'

groupchat = autogen.GroupChat(agents=[user_proxy, assistant, human_readble_assistant], messages=[], max_round=12)
manager = autogen.GroupChatManager(groupchat=groupchat, system_message='Always use "human_readble_assistant" at the end of the conversation.')

In [129]:
# user_proxy.initiate_chat(assistant, message='What is 7 divided by 3?')
user_proxy.initiate_chat(manager, message='What is 7 divided by 3?')

user_proxy (to chat_manager):

What is 7 divided by 3?

--------------------------------------------------------------------------------
assistant (to chat_manager):

***** Suggested function Call: division_calculator *****
Arguments: 
{
  "dividend": 7,
  "divisor": 3
}
********************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION division_calculator...
user_proxy (to chat_manager):

***** Response from calling function "division_calculator" *****
2.3333333333333335
****************************************************************

--------------------------------------------------------------------------------
assistant (to chat_manager):

The result of 7 divided by 3 is approximately 2.33. 

TERMINATE

--------------------------------------------------------------------------------
Ok, if you have any other questions, feel free to ask!human_readble_assistant (to chat_manager):

O

In [131]:
manager.chat_messages
print(human_readble_assistant.system_message)
human_readble_assistant._reply_func_list

You are a logger that should reply with a summary of the conversation so far. 
Use the previous messages when creating the summary. 

Your reply should follow this format: 

[Summary of what has happened so far, aimed at a human end user]

[TERMINATE/CONTINUE]

The end of your response should be either "TERMINATE" or "CONTINUE".
Reply "TERMINATE" if the task has been solved at full satisfaction. 
Otherwise, reply "CONTINUE", or the reason why the task is not solved yet.
Make sure "TERMINATE" and "CONTINUE" are in capital letters.  



[{'trigger': [autogen.agentchat.agent.Agent, None],
  'reply_func': <function __main__.HumanReadbleResponseAgent.produce_response(self, messages: List[Dict] = [], sender=None, config=None) -> Tuple[bool, Union[str, Dict, NoneType]]>,
  'config': None,
  'init_config': None,
  'reset_config': None},
 {'trigger': [autogen.agentchat.agent.Agent, None],
  'reply_func': <function autogen.agentchat.conversable_agent.ConversableAgent.a_check_termination_and_human_reply(self, messages: Optional[List[Dict]] = None, sender: Optional[autogen.agentchat.agent.Agent] = None, config: Optional[Any] = None) -> Tuple[bool, Optional[str]]>,
  'config': None,
  'init_config': None,
  'reset_config': None},
 {'trigger': [autogen.agentchat.agent.Agent, None],
  'reply_func': <function autogen.agentchat.conversable_agent.ConversableAgent.check_termination_and_human_reply(self, messages: Optional[List[Dict]] = None, sender: Optional[autogen.agentchat.agent.Agent] = None, config: Optional[Any] = None) -> Tuple